## 标记静息状态Quiescence
适用于10.16之前跑的旧数据，单独将运动数据输入打印是否有静息状态

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import os
import csv
import cv2
import heapq
from scipy.ndimage import label
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from scipy.ndimage import grey_opening,grey_closing
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from datetime import datetime
from pathlib import Path
from matplotlib.lines import Line2D
import networkx as nx
from scipy.interpolate import interp1d
from scipy import ndimage
from scipy.interpolate import splprep, splev
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d
# 设置字体为 SimHei（黑体）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置字体为 SimHei
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

### 输入总文件夹，读取运动处理文件，输出quies的标记图片
可以用于快速判断哪些文件存在quiescent

### 函数定义

In [ ]:
def cal_low_spd(df, speed_thr = 0.01, dspeed_thr=0.002, ang_thr = 0.02):

    speed = df['speed'].values
    ang_spd = df['ang_velocity'].values
    # 高斯平滑
    sigma = 100   # 平滑参数，可调
    speed_smooth = gaussian_filter1d(speed, sigma=sigma)

    # speed 的导数，再平滑
    dspeed = np.gradient(speed)  # 一阶导数
    dspeed_smooth = gaussian_filter1d(dspeed, sigma=sigma)

    # 头部摆动的角速度
    # dang_velocity = np.gradient(ang_spd)

    # 双条件筛选
    mask = (np.abs(speed_smooth) < speed_thr) & (np.abs(dspeed_smooth) < dspeed_thr) & (np.abs(ang_spd) < ang_thr)
    df['quies'] = 0
    df.loc[mask,'quies'] = 1
    return df
def get_turn_interval(df, col_name):
    # labels打印矩形
    labels = df[col_name].values
    n = len(labels)
    # 找出连续的 label==1 区间
    turn_intervals = []
    in_turn = False
    start = 0

    for i in range(n):
        if labels[i] == 1 and not in_turn:
            # turn开始
            start = i
            in_turn = True
        elif labels[i] != 1 and in_turn:
            # turn结束
            end = i - 1
            turn_intervals.append((start, end))
            in_turn = False

    # 如果最后一个点也是turn状态
    if in_turn:
        turn_intervals.append((start, n - 1))
    return turn_intervals

### 单文件处理

In [ ]:
def plot_quies_single_f(p_f_all, file, p_output):
    # # 重新读取df_merged_2
    f_matched = [f for f in os.listdir(os.path.join(p_f_all, file)) if '_mot_vid_mid.pkl' in f]
    if not len(f_matched):
        f_matched = [f for f in os.listdir(os.path.join(p_f_all, file)) if '_mot_vid_mid.csv' in f]
        df_merged_2 = pd.read_csv(os.path.join(p_f_all, file) +'\\'+f_matched[0])
    else:
        df_merged_2 = pd.read_pickle(os.path.join(p_f_all, file) +'\\'+f_matched[0])
    

    df = cal_low_spd(df_merged_2, speed_thr = 0.01, dspeed_thr=0.002, ang_thr=10)
    quiet_ints = get_turn_interval(df, 'quies')
    # 绘图
    fig,ax = plt.subplots(3,1,figsize=(20,4), sharex=True)
    for start, end in quiet_ints:
        ax[0].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
    # 平滑
    # 平滑turn
    opened = ndimage.binary_closing(df['quies'].values, structure=np.ones(100))
    # 再进行开操作（先腐蚀后膨胀）：去除小的噪声点
    closed = ndimage.binary_opening(opened, structure=np.ones(200))
    # 将结果转换为整数并添加到 DataFrame
    df.loc[:,'quies_pc'] = closed.astype(int)
    quiet_pc_ints = get_turn_interval(df, 'quies_pc')
    for start, end in quiet_pc_ints:
        ax[1].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
    ax[1].set_title('quies_pc')
    ax[2].plot( gaussian_filter1d(df['speed'], sigma=50))
    ax[0].set_title(f'{file}_Quiescence')
    plt.grid(True)
    output_path = p_output+f'\\{file}quis_pc_check.png'  # 替换为你的目标文件夹路径
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    # df_merged_3 = df.copy()
    # df_merged_3.to_pickle(os.path.join(p_f,basename+ '_mot_vid_mid.pkl'))


In [ ]:
p_f_all = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done'
key_word = ''
nokey_word = '*'
path_output = f'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\quies_plot'
files = [f_p for f_p in os.listdir(p_f_all) if ('redo' not in f_p)&
              ('done' not in f_p)&
             os.path.isdir(os.path.join(p_f_all,f_p))&
             (key_word in f_p)&(nokey_word not in f_p)]
print(files)
# files = get_inbtw_file(files, start_str = '20241201', end_str='20250830')
for file in files:
    print(f'=====开始处理{file}======')
    df_p_cor = plot_quies_single_f(p_f_all, file, path_output)

## 标记静息全为0

确定没有静息状态的文件

In [15]:
def add_quiet_trival(p_f):
    folder_name = os.path.basename(os.path.normpath(p_f))

    # 激光段运动数据
    files = [f for f in os.listdir(p_f) if '_mot_midline_cut' in f]
    # 优先找 .pkl
    pkl_files = [f for f in files if f.endswith('.pkl')]
    csv_files = [f for f in files if f.endswith('.csv')]
    if len(pkl_files):
        f_mot_mid_cut = pkl_files[0]   # 如果有多个 .pkl，只取第一个
        df_mot_midline_cut = pd.read_pickle(os.path.join(p_f,f_mot_mid_cut))
    elif len(csv_files):
        f_mot_mid_cut = csv_files[0]   # 没有 .pkl，但有 .csv
        df_mot_midline_cut = pd.read_csv(os.path.join(p_f,f_mot_mid_cut))
    else:
        raise FileNotFoundError("没有找到符合条件的 _mot_midline_cut 文件")
    df_mot_midline_cut['quies'] = 0
    df_mot_midline_cut['quies_pc'] = 0
    f_cut_name = folder_name+'_mot_midline_cut.pkl'
    df_mot_midline_cut.to_pickle(os.path.join(p_f, f_cut_name))

    # 激光对时数据
    mot_mid_pkl = [f for f in os.listdir(p_f) if 'MotionMidlineMatchVol.pkl' in f]
    mot_mid_csv = [f for f in os.listdir(p_f) if 'MotionMidlineMatchVol.csv' in f]
    if len(mot_mid_pkl):
        print('read pickle')
        df_mot_vol = pd.read_pickle(os.path.join(p_f, mot_mid_pkl[0])).reset_index(drop=True)
    elif len(mot_mid_csv):
        df_mot_vol = pd.read_csv(os.path.join(p_f, mot_mid_csv[0]))
    else:
        print('No df_mot_midline')
    df_mot_vol['quies'] = 0
    df_mot_vol['quies_pc'] = 0
    f_vol_name = folder_name+'_MotionMidlineMatchVol.pkl'
    df_mot_vol.to_pickle(os.path.join(p_f, f_vol_name))
    

In [16]:
p_f_all = r'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\done'
key_word = ''
nokey_word = '*'
# path_output = f'Y:\\SZX\\2025_wbi_analysis\\good_WBI\\quies_plot'
files = [f_p for f_p in os.listdir(p_f_all) if ('redo' not in f_p)&
              ('done' not in f_p)&
             os.path.isdir(os.path.join(p_f_all,f_p))&
             (key_word in f_p)&(nokey_word not in f_p)]
print(files)
# files = get_inbtw_file(files, start_str = '20241201', end_str='20250830')
for file in files:
    print(f'=====开始处理{file}======')
    add_quiet_trival(os.path.join(p_f_all,file))

['20241219_0g-ov_05', '20241219_4.5g-ov_06', '20250115_4.5g-24d-ov_08', '20250116_4.5g-ov_05', '20250221_0g-ov-27.5d_6', '20250225_0g-ov-24d_003', '20250225_0g-ov-24d_004', '20250225_4.5g-ov-24d_002', '20250225_4.5g-ov-24d_016', '20250312_4.5g-24d-4h_007', '20250319_1g-23d-4h_001', '20250409_5g-ov-24d_011', '20250422_3g-lg9624-3h-24.5d_004', '20250422_4.5gNa-lg9624-6h-24d_015', '20250425_1g-9624-ov-24h_001', '20250609_lg9624-1g-00225-23.5d-ov_008', '20250609_lg9624-1g-00225-24d-ov_006', '20250805_lg9624-1gNa-24d-ov_009', '20250805_lg9624-1gNa-24d-ov_011', '20250805_lg9624-1gNa-24d-ov_013', '20250823_lg9624-1gNa-24.5d-1h-new_020', '20250823_lg9624-1gNa-24.5d-4h-new_016', '20250823_lg9624-1gNa-24d-1h-new_013', '20250823_lg9624-1gNa-24d-1h-old_006', '20250823_lg9624-1gNa-24d-4h-new_022']
=====开始处理20241219_0g-ov_05======
read pickle
=====开始处理20241219_4.5g-ov_06======
read pickle
=====开始处理20250115_4.5g-24d-ov_08======
read pickle
=====开始处理20250116_4.5g-ov_05======
read pickle
=====开始处理20250